In [1]:
import pandas as pd

In [6]:
df = pd.read_pickle("../../data/dummy.pkl")
df

,state,account length,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,...,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn,total_domestic_minutes,total_domestic_calls,total_domestic_charge,total_charge
0,KS,128,no,yes,25,265.1,110,45.07,197.4,99,...,11.01,10.0,3,2.70,1,0,707.2,300,72.86,75.56
1,OH,107,no,yes,26,161.6,123,27.47,195.5,103,...,11.45,13.7,3,3.70,1,0,611.5,329,55.54,59.24
2,NJ,137,no,no,0,243.4,114,41.38,121.2,110,...,7.32,12.2,5,3.29,0,0,527.2,328,59.00,62.29
3,OH,84,yes,no,0,299.4,71,50.90,61.9,88,...,8.86,6.6,7,1.78,2,0,558.2,248,65.02,66.80
4,OK,75,yes,no,0,166.7,113,28.34,148.3,122,...,8.41,10.1,3,2.73,3,0,501.9,356,49.36,52.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,AZ,192,no,yes,36,156.2,77,26.55,215.5,126,...,12.56,9.9,6,2.67,2,0,650.8,286,57.43,60.10
3329,WV,68,no,no,0,231.1,57,39.29,153.4,55,...,8.61,9.6,4,2.59,3,0,575.8,235,60.94,63.53
3330,RI,28,no,no,0,180.8,109,30.74,288.8,58,...,8.64,14.1,6,3.81,2,0,661.5,258,63.93,67.74
3331,CT,184,yes,no,0,213.8,105,36.35,159.6,84,...,6.26,5.0,10,1.35,2,0,512.6,326,56.18,57.53


In [7]:
df["churn"].value_counts(normalize=True)

0    0.855086
1    0.144914
Name: churn, dtype: float64

The model would only be right 14.5% of the time if the model only guessed every customer would leave.

As we devlop our models, we will be optomizing the recall score of our models. Our strategies We may identify more people as leaving, but in reality they stay. The company will spend some resources to retain the customer. However, that the cost of keeping them engaged with the company far outweighs the cost of identifying someone as staying and in actuality they leave the company. The company would lose a customer and then have to spend more resources to get a new customer in their place.